In [ ]:
import os
os.environ["OPENAI_API_KEY"] .....

In [ ]:
#Question answering over documents consists of four steps:
#1A Create an index

In [ ]:
#pip install pinecone-client tiktoken

In [78]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [79]:
# initialize pinecone
import pinecone
pinecone.init(
    api_key= os.environ['PINECONE_API_KEY'],
    environment="us-west1-gcp-free"  # next to api key in console
)

In [80]:
embeddings = OpenAIEmbeddings()

In [81]:
index_name = "langchain-demo1"  # the name within Pinecone, whether it is a new or old index

In [ ]:
#to embed a new document
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
#New_input = TextLoader(r'C:\Users\User\Downloads\state_of_the_union.txt', encoding='utf-8')
#New_input = TextLoader(r'C:\Users\User\iCloudDrive\00 Motivation or use case or explanation\00 Context and culture and language\2008 Microsoft paper on portfolio management dull skimmed definitions.txt')
#New_input = TextLoader(r'C:\Users\User\iCloudDrive\40 Data\Why American Costs Are So High for rail.txt')
New_input = TextLoader(r'C:\Users\User\iCloudDrive\40 Data\2014 TfL themes and challenges LR.txt')

In [83]:
documents = New_input.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
New_input_as_chunks= text_splitter.split_documents(documents)

In [89]:
# To create a new variable for a vector store associated with an existing or new index
Vector_store2 = Pinecone.from_documents(New_input_as_chunks, embeddings, index_name=index_name)

In [ ]:
Vector_store2.add_texts(["TfL has a lot of resource challenges"])

In [ ]:
# to add docs or text to a particular namespace in an existing index
#Vector_store2.add_texts(["TfL is bit goofy"], namespace='narrative')
#Vector_store2 = Pinecone.from_documents(New_input_as_chunks, embeddings, index_name=index_name, namespace='narrative')

In [97]:
#1C test adding in text w meta data. Can also do documents but not figured out
Vector_store2.add_texts(["TfL has no resource challenges"], metadatas=[{"source": "Ronny"},{"truthfullness": "false"}])

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

['1ab48033-5778-4da9-9d0d-8ffd93b04657']

In [ ]:
#1B basic tests of queries

In [90]:
# if you have not just created the index, load one like this
Vector_store1 = Pinecone.from_existing_index(index_name, embeddings,namespace)

In [99]:
query = "What should i do about asset condition for success"
docs = Vector_store2.similarity_search(query)

In [ ]:
print(docs[0])   #or print(docs[0].page_content)

In [ ]:
#2 Create a Retriever from that index 

In [111]:
retriever_VS2 = Vector_store2.as_retriever()

In [112]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name="gpt-4", temperature=0.0)

In [118]:
#3 Create a question answering chain
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever_VS2)

In [ ]:
#4 Ask questions!

In [115]:
query = "What should i do about asset condition for success"
qa.run(query)

' You should focus on the current TfL themes, the Crossrail SI role and responsibility, the continued cost transformation, the Sponsor, Project, Engineer triangle, novel ways of providing PM resource, and innovation in contracting and big mixed schemes to ensure you are taking the right steps for asset condition to be successful.'